<a href="https://colab.research.google.com/github/alirezash97/Pattern-Recognition-Course/blob/main/HCV_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv'

--2021-03-07 09:34:23--  https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46183 (45K) [application/x-httpd-php]
Saving to: ‘hcvdat0.csv’

hcvdat0.csv         100%[===================>]  45.10K  --.-KB/s    in 0.1s    

2021-03-07 09:34:24 (348 KB/s) - ‘hcvdat0.csv’ saved [46183/46183]



In [ ]:
# !pip install kmeans-smote

In [1]:
import pandas as pd 
import numpy as np


dataset = pd.read_csv('/content/hcvdat0.csv')
dataset.head()

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [2]:
#check for missing values

dataset.isnull().sum()

Unnamed: 0     0
Category       0
Age            0
Sex            0
ALB            1
ALP           18
ALT            1
AST            0
BIL            0
CHE            0
CHOL          10
CREA           0
GGT            0
PROT           1
dtype: int64

In [3]:
# get missing values indexes for ALB, ALP, CHOL and PROT features

ALB_missing_indexes = dataset['ALB'].isnull().to_numpy().nonzero()
ALP_missing_indexes = dataset['ALP'].isnull().to_numpy().nonzero()
ALT_missing_indexes = dataset['ALT'].isnull().to_numpy().nonzero()
CHOL_missing_indexes = dataset['CHOL'].isnull().to_numpy().nonzero()
PROT_missing_indexes = dataset['PROT'].isnull().to_numpy().nonzero()
print('ALB missing values: \n', ALB_missing_indexes)
print('\nALP missing values: \n', ALP_missing_indexes)
print('\nALT missing values: \n', ALT_missing_indexes)
print('\nCHOL missing values: \n', CHOL_missing_indexes)
print('\nPROT missing values: \n', PROT_missing_indexes)

ALB missing values: 
 (array([603]),)

ALP missing values: 
 (array([541, 545, 546, 568, 569, 570, 571, 576, 581, 582, 583, 584, 585,
       590, 592, 603, 613, 614]),)

ALT missing values: 
 (array([540]),)

CHOL missing values: 
 (array([121, 319, 329, 413, 424, 433, 498, 584, 590, 603]),)

PROT missing values: 
 (array([590]),)


In [4]:
def missing_handle_mean(dataframe, column, indexes):
  
  for index in indexes:

    dataframe[column][index] = dataframe[column].mean()
  
  return dataframe


In [5]:
def missing_handle_classmean(dataframe, column, indexes):
  
  for index in indexes:
    
    missing_value_class_values = []
    index_label = dataframe['Category'][index]
    for row_index, row in dataset.iterrows():
      if str(index_label) == str(row['Category']):
        
        missing_value_class_values.append( dataframe[column][row_index] )

    cleanedList = [x for x in missing_value_class_values if str(x) != 'nan']
    dataframe[column][index] = np.mean(cleanedList)
  
  return dataframe

In [6]:
def preprocessing(df, missing_handler=None):

  for row_index, row in df.iterrows():

    # Sex Handler
    if row['Sex'] == 'm':
      df['Sex'][row_index] = 1
    elif row['Sex'] == 'f':
      df['Sex'][row_index] = 2
    else:
      pass

    # Category to numerical 
    if row['Category'] == '0=Blood Donor':
      df['Category'][row_index] = 0
    elif row['Category'] == '0s=suspect Blood Donor':
      df['Category'][row_index] = 1
    elif row['Category'] == '1=Hepatitis':
      df['Category'][row_index] = 2
    elif row['Category'] == '2=Fibrosis':
      df['Category'][row_index] = 3
    elif row['Category'] == '3=Cirrhosis':
      df['Category'][row_index] = 4
    else:
      pass

  
  if missing_handler == 'Column_Mean':
    df = missing_handle_mean(df, 'ALB', ALB_missing_indexes[0])
    df = missing_handle_mean(df, 'ALP', ALP_missing_indexes[0])
    df = missing_handle_mean(df, 'ALT', ALT_missing_indexes[0])
    df = missing_handle_mean(df, 'CHOL', CHOL_missing_indexes[0])
    df = missing_handle_mean(df, 'PROT', PROT_missing_indexes[0])
  elif missing_handler == 'Class_Mean':
    df = missing_handle_classmean(df, 'ALB', ALB_missing_indexes[0])
    df = missing_handle_classmean(df, 'ALP', ALP_missing_indexes[0])
    df = missing_handle_classmean(df, 'ALT', ALT_missing_indexes[0])
    df = missing_handle_classmean(df, 'CHOL', CHOL_missing_indexes[0])
    df = missing_handle_classmean(df, 'PROT', PROT_missing_indexes[0])
  else:
    pass

  
  return df


In [7]:
dataset = preprocessing(dataset, missing_handler='Class_Mean')

# checking fo missing values after preprocessing
dataset.isnull().sum()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-p

Unnamed: 0    0
Category      0
Age           0
Sex           0
ALB           0
ALP           0
ALT           0
AST           0
BIL           0
CHE           0
CHOL          0
CREA          0
GGT           0
PROT          0
dtype: int64

In [8]:
def split_dataset(df, train_percentage=0.8, test_percentage=0.2):



  train = df.sample(frac=(1-test_percentage),random_state=200) #random state is a seed value
  test = df.drop(train.index)

  return train, test


In [9]:
trainset, testset = split_dataset(dataset, 0.8, 0.2)

In [10]:
from sklearn.preprocessing import LabelBinarizer
from scipy import sparse

# dataframe to numpy array 
x_train = trainset.drop(['Category', 'Unnamed: 0'], axis = 1).values
y_train = trainset.loc[:, 'Category'].values.astype('int')

x_test = testset.drop(['Category', 'Unnamed: 0'], axis = 1).values
y_test = testset.loc[:, 'Category'].values.astype('int')


In [11]:
print('x_train shapes: ', x_train.shape)
print('y_train shapes: ', y_train.shape)
print('x_test shapes: ', x_test.shape)
print('x_test shapes: ', x_test.shape)

x_train shapes:  (492, 12)
y_train shapes:  (492,)
x_test shapes:  (123, 12)
x_test shapes:  (123, 12)


In [12]:
print('number of <Blood Donor> samples: ', np.count_nonzero(y_train == 0))
print('number of <suspect Blood Donor> samples in 2th class: ', np.count_nonzero(y_train == 1))
print('number of <Hepatitis> samples in 2th class: ', np.count_nonzero(y_train == 2))
print('number of <Cirrhosis> samples in 2th class: ', np.count_nonzero(y_train == 3))

number of <Blood Donor> samples:  425
number of <suspect Blood Donor> samples in 2th class:  6
number of <Hepatitis> samples in 2th class:  21
number of <Cirrhosis> samples in 2th class:  17


In [20]:
# handle imbalanced data 
from kmeans_smote import KMeansSMOTE

kmeans_smote = KMeansSMOTE(
    kmeans_args={
        'n_clusters': 100
    },
    smote_args={
        'k_neighbors': 5
    }
)

X_train_resampled, y_train_resampled = kmeans_smote.fit_sample(x_train, y_train)


In [21]:
print('new msamples added, x_train shape : ', X_train_resampled.shape)
print('new msamples added, y_train shape : ', y_train_resampled.shape)

new msamples added, x_train shape :  (2125, 12)
new msamples added, y_train shape :  (2125,)


In [22]:
print('number of <Blood Donor> samples: ', np.count_nonzero(y_train_resampled == 0))
print('number of <suspect Blood Donor> samples: ', np.count_nonzero(y_train_resampled == 1))
print('number of <Hepatitis> samples: ', np.count_nonzero(y_train_resampled == 2))
print('number of <Cirrhosis> samples: ', np.count_nonzero(y_train_resampled == 3))

number of <Blood Donor> samples:  425
number of <suspect Blood Donor> samples:  425
number of <Hepatitis> samples:  425
number of <Cirrhosis> samples:  425


In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = SVC(kernel='linear', decision_function_shape='ovr')
clf.fit(X_train_resampled, y_train_resampled)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [24]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(x_test)
print(y_pred)
print(y_test)
confusion_matrix(y_test, y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 3 2
 0 2 2 2 3 3 4 4 4 2 4 4]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2
 2 3 3 3 3 4 4 4 4 4 4 4]


array([[107,   0,   1,   0,   0],
       [  0,   1,   0,   0,   0],
       [  1,   0,   1,   1,   0],
       [  0,   0,   3,   1,   0],
       [  0,   0,   1,   1,   5]])

In [25]:
clf.score(x_test, y_test)

0.9349593495934959